<a href="https://colab.research.google.com/github/afifaniks/TomatoLeafDiseaseDetection/blob/master/Leaf_Disease_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **TLNet**: Tomato Leaf Disease Detection
Author: [Afif Al Mamun](https://afifaniks.me) <br>
Date: September, 2020

# Loading Data

In [38]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [39]:
!cp 'drive/My Drive/Leaf Disease/Dataset/Train_Dataset.zip' .

In [ ]:
!unzip Train_Dataset.zip

Archive:  Train_Dataset.zip
replace Data_3/Test/Tomato_Bacterial_Spot/0ab9c705-f29e-45ac-b786-9549b3c38f16___GCREC_Bact.Sp 3223.JPG? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

# Importing Modules

In [ ]:
!pip install extra_keras_metrics

In [ ]:
%tensorflow_version 1.15
import os
import matplotlib.pyplot as plt
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Activation, Dropout, Flatten, Dense
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras.layers.normalization import BatchNormalization
from keras import backend as K
import keras
from keras.preprocessing import image
import numpy as np
import extra_keras_metrics as ekm
from keras.utils import plot_model
import json
from google.colab import files
import seaborn as sn
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
import pandas as pd

# Data Preparation

In [ ]:
data_folder = 'Data_3/Train'
directory = os.listdir(data_folder)

file_counts = {}
for d in directory:
    file_counts[d] = len(os.listdir(os.path.join(data_folder, d)))
    
# Plot bar chart
plt.figure(figsize=(8,4))
plt.bar(file_counts.keys(), file_counts.values())
plt.xticks(rotation='vertical')
plt.show()

In [ ]:
data_folder = 'Data_3/Test'
directory = os.listdir(data_folder)

file_counts = {}
for d in directory:
    file_counts[d] = len(os.listdir(os.path.join(data_folder, d)))
    
# Plot bar chart
plt.figure(figsize=(8,4))
plt.bar(file_counts.keys(), file_counts.values())
plt.xticks(rotation='vertical')
plt.show()

In [ ]:
image = plt.imread('Data_3/Train/Tomato_Bacterial_Spot/0045ba29-ed1b-43b4-afde-719cc7adefdb___GCREC_Bact.Sp 6254.JPG')
image.shape

In [ ]:
# Image preprocessing
img_height = image.shape[0]
img_width = image.shape[1]

train_data_dir = 'Data_3/Train'
test_data_dir = 'Data_3/Test'

epochs = 50
batch_size = 20
batch_dim = -1

if K.image_data_format() == 'channels_first':
    input_shape = (3, img_height, img_width)
    batch_dim = 1
else:
    input_shape = (img_height, img_width, 3)

train_datagen = ImageDataGenerator(rescale=1./255, shear_range=0.2, zoom_range=0.2, rotation_range=0.2, horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(train_data_dir, target_size=(img_height, img_width), batch_size=batch_size, class_mode='categorical')
test_generator = train_datagen.flow_from_directory(test_data_dir, target_size=(img_height, img_width), batch_size=batch_size, class_mode='categorical')

In [ ]:
train_size = len(train_generator.labels)
test_size = len(test_generator.labels)

# Model Preparation

In [ ]:
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=input_shape, bias_regularizer=keras.regularizers.l1_l2(l1=0.1, l2=0.01), activation='relu'))
model.add(BatchNormalization(axis=batch_dim))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3), bias_regularizer=keras.regularizers.l1_l2(l1=0.1, l2=0.01), activation='relu'))
model.add(BatchNormalization(axis=batch_dim))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128, (3, 3), bias_regularizer=keras.regularizers.l1_l2(l1=0.1, l2=0.01), activation='relu'))
model.add(BatchNormalization(axis=batch_dim))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128, (3, 3), bias_regularizer=keras.regularizers.l1_l2(l1=0.1, l2=0.01), activation='relu'))
model.add(BatchNormalization(axis=batch_dim))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3), bias_regularizer=keras.regularizers.l1_l2(l1=0.1, l2=0.01), activation='relu'))
model.add(BatchNormalization(axis=batch_dim))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(BatchNormalization(axis=batch_dim))

model.add(Dense(64, activation='relu'))
model.add(BatchNormalization(axis=batch_dim))
model.add(Dropout(0.2))

model.add(Dense(8))
model.add(Activation(activation='softmax'))

model.summary()

In [ ]:
plot_model(model, to_file='model.png')

In [ ]:
callbacks = [
    # EarlyStopping(patience=10, verbose=1),
    ReduceLROnPlateau(factor=0.1, patience=5, min_lr=0.000001, verbose=1),
    ModelCheckpoint('plants.h5', verbose=1, save_best_only=True, save_weights_only=True)
]

In [ ]:
metrics=["accuracy", ekm.precision, ekm.recall]
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=["accuracy"])
results = model.fit_generator(train_generator, epochs=epochs, steps_per_epoch=np.ceil(train_size/batch_size), validation_data=test_generator, validation_steps=np.ceil(test_size/batch_size), callbacks=callbacks)

# Observations

In [ ]:
plt.figure(figsize=(8, 5))
plt.title("Training Summary")
plt.xlim(0, 50)
plt.ylim(0, 1)
plt.plot(results.history['accuracy'], label="acc")
plt.plot(results.history['loss'], label="loss")
plt.plot(results.history['precision'], label="precision")
plt.plot(results.history['recall'], label="recall")
plt.xlabel("Epochs")
plt.ylabel("Value")
plt.legend();

In [ ]:
plt.figure(figsize=(8, 5))
plt.title("Validation Summary")
plt.xlim(0, 50)
plt.ylim(0, 1)
plt.plot(results.history['val_accuracy'], label="val_acc")
plt.plot(results.history['val_loss'], label="val_loss")
plt.plot(results.history['val_precision'], label="val_precision")
plt.plot(results.history['val_recall'], label="val_recall")
plt.xlabel("Epochs")
plt.ylabel("Value")
plt.legend();

# Downloads

In [ ]:
model_history = results.history

In [ ]:
# Download history
with open("history.json", "w") as f:
    json.dump(str(model_history), f)
files.download("history.json")
# Download model
files.download(plants.h5)

# Model Performance Evaluation

In [ ]:
!cp 'drive/My Drive/Leaf Disease/Dataset/Test_Dataset.zip' .
!unzip Test_Dataset.zip

In [ ]:
preds = []

test_data_dir = 'Test_Dataset'

dirs = sorted(os.listdir(test_data_dir))

for class_index in range(len(dirs)):
  class_path = os.path.join(test_data_dir, dirs[class_index])
  files = sorted(os.listdir(class_path))

  for f in files:
    image_path = os.path.join(class_path, f)
    image_pred = image.load_img(image_path, target_size=(img_height, img_width))

    image_pred = image.img_to_array(image_pred) / 256
    image_pred = np.expand_dims(image_pred, axis=0)

    result = model.predict_classes(image_pred)

    preds.append(result[0])
  
  class_index += 1


In [ ]:
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(test_data_dir, target_size=(img_height, img_width), class_mode='categorical')
target_names = list(test_generator.class_indices.keys())
target_names

### Confusion Matrix

In [ ]:
con_mat = confusion_matrix(test_generator.classes, preds)
df_cm = pd.DataFrame(con_mat, columns=target_names, index=target_names)
sn.heatmap(df_cm, annot=True)

### Classification Report

In [ ]:
print(classification_report(test_generator.classes, preds, target_names=target_names))